In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

#try:
    # %tensorflow_version only exists in Colab.
#    %tensorflow_version 2.0
#except Exception:
#    pass

# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras
# Helper libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.utils import shuffle
from sklearn.metrics import confusion_matrix, multilabel_confusion_matrix

tf.config.experimental.set_visible_devices([], 'GPU')

AUTOTUNE = tf.data.experimental.AUTOTUNE
import IPython.display as display


import os
from os import listdir
from os.path import isfile, join
import pickle
import random
#importing module
import sys
sys.path.insert(0, '../data')
from datahandler_cascading import create_dataset

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt



print(tf.__version__)
#base_model = model.output

2.0.0


In [2]:
with open('../data/train_test_split_hierarchical.pkl', 'rb') as infile:
    train_x, train_y, train_y_, val_x, val_y, val_y_, test_x, test_y, test_y_ = pickle.load(infile)

In [9]:
train_generator = create_dataset(train_x, train_y, train_y_)
val_generator = create_dataset(val_x, val_y, val_y_)
test_generator = create_dataset(test_x, test_y, test_y_, BATCH_SIZE=1)

In [14]:
class Coarse(tf.keras.layers.Layer):
    def __init__(self, filters, shape=(224,224,3), kernel_size=3,
                 pool_size=(2,2), activation='relu', padding='same'):
        super(Coarse, self).__init__()
        self.input_layer = tf.keras.layers.InputLayer(
            #input_shape = shape,
        )
        self.conv_layer = tf.keras.layers.Conv2D(
            filters = filters[0], 
            kernel_size = kernel_size,
            activation = activation,
            )
        self.nn_pooling = tf.keras.layers.MaxPooling2D(
            pool_size = pool_size,
            padding = padding,
            )
        self.nn_dropout = tf.keras.layers.Dropout(0.2)
        self.nn_batchnorm = tf.keras.layers.BatchNormalization(axis=-1)
        self.flat = tf.keras.layers.Flatten()#input_shape = (13, 13, 64))
        self.dense = tf.keras.layers.Dense(filters[1], activation='relu')
        self.out = tf.keras.layers.Dense(filters[2], activation='softmax')

        
        
    def call(self, input_features):
        activation = self.input_layer(input_features)
        activation = self.conv_layer(activation)
        activation = self.nn_batchnorm(activation)
        activation = self.nn_pooling(activation)
        print(input_features.shape,activation.shape)
        activation = self.flat(activation)
        activation = self.dense(activation)
        return self.out(activation)


In [15]:
def macro_f1(y, y_hat, thresh=0.5):
    y_pred = tf.cast(tf.greater(y_hat, thresh), tf.float32)
    tp = tf.cast(tf.math.count_nonzero(y_pred * y, axis=0), tf.float32)
    fp = tf.cast(tf.math.count_nonzero(y_pred * (1 - y), axis=0), tf.float32)
    fn = tf.cast(tf.math.count_nonzero((1 - y_pred) * y, axis=0), tf.float32)
    f1 = 2*tp / (2*tp + fn + fp + 1e-16)
    macro_f1 = tf.reduce_mean(f1)
    return macro_f1

def accuracy(y, y_hat):
    y_pred = tf.cast(tf.greater(y_hat, 0.5), tf.float32)
    tp = tf.cast(tf.math.count_nonzero(y_pred * y, axis=0), tf.float32)
    fp = tf.cast(tf.math.count_nonzero(y_pred * (1 - y), axis=0), tf.float32)
    fn = tf.cast(tf.math.count_nonzero((1 - y_pred) * y, axis=0), tf.float32)
    tn = tf.cast(tf.math.count_nonzero((1 - y_pred) * (1 - y), axis=0), tf.float32)
    accuracy = (tp + tn) / (tp + tn + fn + fp)
    accuracy = tf.reduce_mean(accuracy)
    return accuracy

def precision(y, y_hat):
    y_pred = tf.cast(tf.greater(y_hat, 0.5), tf.float32)
    tp = tf.cast(tf.math.count_nonzero(y_pred * y, axis=0), tf.float32)
    fp = tf.cast(tf.math.count_nonzero(y_pred * (1 - y), axis=0), tf.float32)
    pres = tp / (tp + fp + 1e-16)
    precision = tf.reduce_mean(pres)
    return precision

def recall(y, y_hat):
    y_pred = tf.cast(tf.greater(y_hat, 0.5), tf.float32)
    tp = tf.cast(tf.math.count_nonzero(y_pred * y, axis=0), tf.float32)
    fn = tf.cast(tf.math.count_nonzero((1 - y_pred) * y, axis=0), tf.float32)
    rec = tp / (tp + fn + 1e-16)
    recall = tf.reduce_mean(rec)
    return recall

In [18]:
model_c = keras.Sequential([
    Coarse([64, 128, 15])
])

  
model_c.compile(optimizer='adam',
                loss='binary_crossentropy',
                metrics=[accuracy, recall, precision, macro_f1])

for i in range(len(model_c.layers)-1):
    model_c.layers[i].set_weights(model.layers[i].get_weights())

In [20]:
index = 30
step = 5
stop = 50

while index < stop:
    model_c.fit(train_generator,
          epochs=index+step,
          verbose=1, initial_epoch=index,
          validation_data = val_generator
          )
    index += step

(None, 224, 224, 3) (None, 111, 111, 64)
Epoch 31/35
      1/Unknown - 0s 159ms/step

ValueError: Found a sample_weight array with shape (None, 141). In order to use timestep-wise sample weights, you should specify sample_weight_mode="temporal" in compile(). If you just mean to use sample-wise weights, make sure your sample_weight array is 1D.

In [21]:
#constructing fine classifiers
class Fine(tf.keras.layers.Layer):
    def __init__(self, filters, shape=(224,224,1), kernel_size=3,
                 pool_size=(2,2), activation='relu', padding='same'):
        super(Fine, self).__init__()
        self.input_layer = tf.keras.layers.InputLayer(
            #input_shape = shape,
        )
        self.conv_layer = tf.keras.layers.Conv2D(
            filters = filters[0], 
            kernel_size = kernel_size,
            activation = activation,
            )
        self.nn_pooling = tf.keras.layers.MaxPooling2D(
            pool_size = pool_size,
            padding = padding,
            )
        self.nn_dropout = tf.keras.layers.Dropout(0.2)
        self.nn_batchnorm = tf.keras.layers.BatchNormalization(axis=-1)
        self.flat = tf.keras.layers.Flatten()#input_shape = (13, 13, 64))
        self.dense = tf.keras.layers.Dense(filters[1], activation='relu')
        self.out = tf.keras.layers.Dense(filters[2], activation='softmax')

        
        
    def call(self, input_features):
        activation = self.input_layer(input_features)
        activation = self.conv_layer(activation)
        activation = self.nn_batchnorm(activation)
        activation = self.nn_pooling(activation)
        print(input_features.shape,activation.shape)
        activation = self.flat(activation)
        activation = self.dense(activation)
        return self.out(activation)

#base_model = model.output

#train_images = train_images.reshape(train_images.shape[0], 28, 28, 1).astype('float32')
#test_images = test_images.reshape(test_images.shape[0], 28, 28, 1).astype('float32')

def fine_model():
    model_fine = keras.Sequential([
        #base_model,
        Fine([64, 128, 141])
    ])

  
    model_fine.compile(optimizer='adam',
                    loss='sparse_categorical_crossentropy',
                    metrics=['accuracy'])

    for i in range(len(model_fine.layers)-1):
        model_fine.layers[i].set_weights(model.layers[i].get_weights())
    return model_fine

In [22]:
fine_models = {'models' : [{} for i in range(2)], 'yhf' : [{} for i in range(2)]}
for i in range(2):
    model_i = fine_model()
    fine_models['models'][i] = model_i

In [23]:
def get_error(y,yh):
    # Threshold 
    yht = np.zeros(np.shape(yh))
    yht[np.arange(len(yh)), yh.argmax(1)] = 1
    # Evaluate Error
    error = np.count_nonzero(np.count_nonzero(y-yht,1))/len(y)
    return error

In [24]:
#training fine classifiers on corresponding data
for cat in range(2):
    index= 0
    step = 5
    stop = 30
    
    while index < stop:
        fine_models['models'][cat].fit(train_images, train10_labels[cat], 
          epochs=index+step,
          verbose=1, initial_epoch=index,
          #validation_data = valgenlist[cat]
          )
        index += step
    
    #fine_models['models'][cat].compile(optimizer=sgd_fine, loss='categorical_crossentropy', metrics=['accuracy'])
    #stop = 50

    #while index < stop:
     #   fine_models['models'][cat].fit_generator(traingenlist[cat],
      #    epochs=index+step,
       #   verbose=1, initial_epoch=index,
        #  validation_data = valgenlist[cat]
         # )
       # index += step
        
    #check: compilation needed??
    
    yh_f = fine_models['models'][cat].predict(test_images)
    print('Fine Classifier '+str(cat)+' Error: ')
    #str(get_error(y_val[ix_v],yh_f)))

NameError: name 'train_images' is not defined

In [7]:
coarse_predictions=model_c.predict(test_images)
#coarse_predictions=model_c.predict(img)
#predictions from #(coarse categories) fine classifiers
fine_predictions = [] #dim:  n_classes_coarse X n_images_predict X n_classes_fine
for c in range(2):
    fine_predictions.append(fine_models['models'][c].predict(test_images))

NameError: name 'model_c' is not defined

In [8]:
prediction_size = len(coarse_predictions)
predictions = []
for img in range(prediction_size):
    proba = [0]*10
    for finec in range(10):
        for coarsec in range(2):
            proba[finec] += coarse_predictions[img][coarsec]*fine_predictions[coarsec][img][finec]
    predicted = np.argmax(proba)
    predictions.append(predicted)

NameError: name 'coarse_predictions' is not defined

In [9]:
def eval_hdcnn(X, y):
    yh = np.zeros(np.shape(y))
    
    yh_s = model.predict(X, batch_size=32)
    
    print('Single Classifier Error: '+str(get_error(y,yh_s)))
    
    yh_c = model_c.predict(X, batch_size=32)
    y_c = np.dot(y,fine2coarse)
    
    print('Coarse Classifier Error: '+str(get_error(y_c,yh_c)))

    for i in range(2):
        if i%5 == 0:
            print("Evaluating Fine Classifier: ", str(i))
        #fine_models['yhf'][i] = fine_models['models'][i].predict(X, batch_size=batch)
        yh += np.multiply(yh_c[:,i].reshape((len(y)),1), fine_models['yhf'][i])
    
    print('Overall Error: '+str(get_error(y,yh)))
    return yh